In [1]:
import requests

In [6]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

### Be-a-u-ti-ful Soup Demos

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [11]:
list(soup.children)

[u'html',
 u'\n',
 <html>\n<head>\n<title>A simple example page</title>\n</head>\n<body>\n<p>Here is some simple content for this page.</p>\n</body>\n</html>]

In [12]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [14]:
html = list(soup.children)[2]
body = list(html.children)[3]
p = list(body.children)[1]
p.get_text()

u'Here is some simple content for this page.'

In [16]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')[0].get_text()

u'Here is some simple content for this page.'

In [17]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Partly cloudy, with a low around 49. West wind 8 to 17 mph, with gusts as high as 22 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Partly cloudy, with a low around 49. West wind 8 to 17 mph, with gusts as high as 22 mph. "/>
 </p>
 <p class="short-desc">
  Partly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 49 °F
 </p>
</div>


In [18]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Tonight
Partly Cloudy
Low: 49 °F


In [19]:
img = tonight.find("img")
desc = img['title']

print(desc)


Tonight: Partly cloudy, with a low around 49. West wind 8 to 17 mph, with gusts as high as 22 mph. 


In [20]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

[u'Tonight',
 u'Wednesday',
 u'WednesdayNight',
 u'Thursday',
 u'ThursdayNight',
 u'Friday',
 u'FridayNight',
 u'Saturday',
 u'SaturdayNight']

In [21]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

[u'Partly Cloudy', u'Partly Sunny', u'ChanceShowers', u'Sunny', u'Clear', u'Sunny', u'Clear', u'Sunny', u'Mostly Clear']
[u'Low: 49 \xb0F', u'High: 58 \xb0F', u'Low: 49 \xb0F', u'High: 62 \xb0F', u'Low: 50 \xb0F', u'High: 66 \xb0F', u'Low: 51 \xb0F', u'High: 69 \xb0F', u'Low: 52 \xb0F']
[u'Tonight: Partly cloudy, with a low around 49. West wind 8 to 17 mph, with gusts as high as 22 mph. ', u'Wednesday: Partly sunny, with a high near 58. West wind 11 to 15 mph, with gusts as high as 20 mph. ', u'Wednesday Night: A 40 percent chance of showers.  Mostly cloudy, with a low around 49. West southwest wind 8 to 16 mph, with gusts as high as 21 mph.  New precipitation amounts between a tenth and quarter of an inch possible. ', u'Thursday: Sunny, with a high near 62. West wind 9 to 14 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 28 mph. ', u'Thursday Night: Clear, with a low around 50. West southwest wind 15 to 20 mph decreasing to 7 to 12 mph after midnight. Win

In [22]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,desc,period,short_desc,temp
0,"Tonight: Partly cloudy, with a low around 49. ...",Tonight,Partly Cloudy,Low: 49 °F
1,"Wednesday: Partly sunny, with a high near 58. ...",Wednesday,Partly Sunny,High: 58 °F
2,Wednesday Night: A 40 percent chance of shower...,WednesdayNight,ChanceShowers,Low: 49 °F
3,"Thursday: Sunny, with a high near 62. West win...",Thursday,Sunny,High: 62 °F
4,"Thursday Night: Clear, with a low around 50. W...",ThursdayNight,Clear,Low: 50 °F
5,"Friday: Sunny, with a high near 66.",Friday,Sunny,High: 66 °F
6,"Friday Night: Clear, with a low around 51.",FridayNight,Clear,Low: 51 °F
7,"Saturday: Sunny, with a high near 69.",Saturday,Sunny,High: 69 °F
8,"Saturday Night: Mostly clear, with a low aroun...",SaturdayNight,Mostly Clear,Low: 52 °F


In [23]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    49
1    58
2    49
3    62
4    50
5    66
6    51
7    69
8    52
Name: temp_num, dtype: object

In [24]:
weather["temp_num"].mean()

56.22222222222222

In [25]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool